In [1]:
using SymPy
using OffsetArrays
using Latexify

@syms q0 q1 q2
@syms m::(real, positive) t::(real, positive)
q0′, q1′, q2′, qf′, qf = symbols("q_{0}^{\\prime}, q_{1}^{\\prime}, q_{2}^{\\prime}, q_{F}^{\\prime}, q_F")
t0, t1, t2, tf = symbols("t_0, t_1, t_2, t_F", real = true, positive = true)

function extractPolynomial(expr, arg)
    expr isa Sym ? nothing : expr = sympify(1)
    expr = expand(expr)
    polyDegree = Int(degree(expr, arg))
    extractOneSet = []
    for i in 0:polyDegree
        temp = expr.coeff(arg, i)
        push!(extractOneSet, temp)
    end
    polyExt = OffsetVector(extractOneSet, 0:polyDegree)
    return polyExt
end

extractPolynomial (generic function with 1 method)

In [2]:
# Simplifying the product of propagators in eq. 2.25
idenFunc = IM*m/(2 * (tf - t)) * (qf - q1)^2 + IM*m/(2*t) * (q1 - q0)^2

idenPoly = extractPolynomial(idenFunc, q1)
idenPolyA = - idenPoly[2]
idenPolyB = - idenPoly[1]
idenPolyC = - idenPoly[0]

display(-idenPolyA)
display(-idenPolyB)
display(-idenPolyC)

# (idenPolyB^2 / (4 * idenPolyA) - idenPolyC).simplify()

    ⅈ⋅m        ⅈ⋅m
──────────── + ───
-2⋅t + 2⋅t_F   2⋅t

  ⅈ⋅m⋅q₀    2⋅ⅈ⋅m⋅q_F  
- ────── - ────────────
    t      -2⋅t + 2⋅t_F

      2            2  
ⅈ⋅m⋅q₀      ⅈ⋅m⋅q_F   
─────── + ────────────
  2⋅t     -2⋅t + 2⋅t_F

In [7]:
# Degree one Gaussian integral

# deg1coef -> The coefficient from the Gaussian integral degree one. 
deg1coef = - idenPolyB / 2 * sqrt(PI/(idenPolyA^3))
display(deg1coef.simplify())

# Multiply sqrt(2*PI*IM*t/m) * m/(2*PI)*sqrt(1/(t-tf)) because propagator
deg1coef *= sqrt(m*tf / (2 * PI * IM * t * (tf - t)))
deg1coef = deg1coef.simplify().powsimp(force = true)
display(deg1coef)

# Degree two Gaussian integral

deg2coef = 1//4 * sqrt(PI/idenPolyA^5) * (2*idenPolyA + idenPolyB^2)
deg2coef = deg2coef.simplify()
display(deg2coef)

# deg2coef *= sqrt(2*PI*IM*t/m) * m/(2*PI)*sqrt(1/(t-tf))
# deg2coef = deg2coef.simplify(force = true)

              ______________                        
             ╱            3                         
√2⋅ⅈ⋅√π⋅√t⋅╲╱  ⅈ⋅(t - t_F)  ⋅(-q₀⋅(t - t_F) + q_F⋅t)
────────────────────────────────────────────────────
                      3/2                           
                √m⋅t_F   ⋅(t - t_F)                 

         3/2    ______________                        
⎛   ⅈ   ⎞      ╱            3                         
⎜───────⎟   ⋅╲╱  ⅈ⋅(t - t_F)  ⋅(-q₀⋅(t - t_F) + q_F⋅t)
⎝t - t_F⎠                                             
──────────────────────────────────────────────────────
                         t_F                          

            _______________                                                   
           ╱             5  ⎛                          2      2               
√2⋅√π⋅√t⋅╲╱  -ⅈ⋅(t - t_F)  ⋅⎝- m⋅(q₀⋅(t - t_F) - q_F⋅t)  + ⅈ⋅t ⋅(t - t_F) - ⅈ⋅
──────────────────────────────────────────────────────────────────────────────
                                    3/2    5/2          2                     
                                   m   ⋅t_F   ⋅(t - t_F)                      

             
           2⎞
t⋅(t - t_F) ⎠
─────────────
             
             

In [4]:
# The component of IP3, w.r.t. time.
IP3cGr = (tf - t) * (q0′*(tf - t) + qf′*t) * (q0*(tf - t) + qf*t)
IP3cGr = IP3cGr.expand()
IP3cGrExt = extractPolynomial(IP3cGr, t)
display.(IP3cGrExt)

IP3cG = integrate(IP3cGr, t)
IP3cG.subs(t, tf).simplify()

                      3
q₀⋅q_{0}__{\prime}⋅t_F 

                          2                         2                         
- 3⋅q₀⋅q_{0}__{\prime}⋅t_F  + q₀⋅q_{F}__{\prime}⋅t_F  + q_F⋅q_{0}__{\prime}⋅t_

 2
F 

3⋅q₀⋅q_{0}__{\prime}⋅t_F - 2⋅q₀⋅q_{F}__{\prime}⋅t_F - 2⋅q_F⋅q_{0}__{\prime}⋅t_
F + q_F⋅q_{F}__{\prime}⋅t_F

-q₀⋅q_{0}__{\prime} + q₀⋅q_{F}__{\prime} + q_F⋅q_{0}__{\prime} - q_F⋅q_{F}__{\
prime}

   4                                                                          
t_F ⋅(3⋅q₀⋅q_{0}__{\prime} + q₀⋅q_{F}__{\prime} + q_F⋅q_{0}__{\prime} + q_F⋅q_
──────────────────────────────────────────────────────────────────────────────
                                             12                               

              
{F}__{\prime})
──────────────
              

In [5]:
IP1cGr = sqrt(tf - t) * (m * (q0 * (tf - t) + qf * t)^2 + IM * t * tf * (tf - t))
IP1cGr = IP1cGr.expand()

IP1cG = integrate(IP1cGr, (t, 0, tf))
IP1cG.simplify()

     7/2 ⎛       2                        2          ⎞
2⋅t_F   ⋅⎝15⋅m⋅q₀  + 12⋅m⋅q₀⋅q_F + 8⋅m⋅q_F  + 6⋅ⅈ⋅t_F⎠
──────────────────────────────────────────────────────
                         105                          